<a href="https://colab.research.google.com/github/gnoziere/cs230-ak-troll-detector/blob/main/Troll_Detector_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install transformers

     |████████████████████████████████| 3.1 MB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 21.8 MB/s 
     |████████████████████████████████| 596 kB 29.6 MB/s 
     |████████████████████████████████| 895 kB 44.3 MB/s 
     |████████████████████████████████| 59 kB 7.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import AutoModel, AutoTokenizer

In [ ]:
device = "cuda"

In [ ]:
# Define paths
positive_data_path = "/content/drive/MyDrive/turkey_052020_tweets_csv_hashed_2011.csv"
negative_data_path = "/content/drive/MyDrive/turkey_052020_tweets_csv_hashed_2020_01.csv"
model_path = "dbmdz/bert-base-turkish-cased"

In [ ]:
positive_raw_data = pd.read_csv(positive_data_path)
positive_raw_data["label"] = 1.0

negative_raw_data = pd.read_csv(negative_data_path)
negative_raw_data["label"] = 0.0

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
positive_raw_data.columns

Index(['tweetid', 'userid', 'user_display_name', 'user_screen_name',
       'user_reported_location', 'user_profile_description',
       'user_profile_url', 'follower_count', 'following_count',
       'account_creation_date', 'account_language', 'tweet_language',
       'tweet_text', 'tweet_time', 'tweet_client_name', 'in_reply_to_userid',
       'in_reply_to_tweetid', 'quoted_tweet_tweetid', 'is_retweet',
       'retweet_userid', 'retweet_tweetid', 'latitude', 'longitude',
       'quote_count', 'reply_count', 'like_count', 'retweet_count', 'hashtags',
       'urls', 'user_mentions', 'label'],
      dtype='object')

In [ ]:
positive_raw_data.shape

(35100, 31)

In [ ]:
negative_raw_data.shape

(859516, 31)

In [ ]:
merged_data = pd.concat([positive_raw_data, negative_raw_data])
merged_data = merged_data.sample(frac=1)

In [ ]:
merged_data

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,account_language,tweet_language,tweet_text,tweet_time,tweet_client_name,in_reply_to_userid,in_reply_to_tweetid,quoted_tweet_tweetid,is_retweet,retweet_userid,retweet_tweetid,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,label
112997,1220934027017891840,2256740149,Hikmet Şahin,25Davam,Oltu/Erzurum,Mevkimiz KABİR\nMakamı ALLAH C.C verir\n2071 G...,NaN,27856,13498,2013-12-21,tr,tr,RT @hYDoKJWSCYXElDSC5UwMJWfNaC4ft0LrhOkfl6dIFw...,2020-01-25 04:58,Twitter for Android,NaN,NaN,NaN,True,2256740149,1.220791e+18,absent,absent,0,0,0,0,[],[],['hYDoKJWSCYXElDSC5UwMJWfNaC4ft0LrhOkfl6dIFw='...,0.0
657517,1215655706835243015,877965285516615682,Güldane53,guldane53,NaN,"kimin yarsına merhem olursan ol,iyileştiği an ...",NaN,16759,13917,2017-06-22,tr,tr,RT @zemzem35: Rabbena Atina fiddünya\nVe fil a...,2020-01-10 15:24,Twitter for iPhone,NaN,NaN,NaN,True,NaN,1.215478e+18,absent,absent,0,0,0,0,[],[],['476102546'],0.0
737256,1218238684337725445,UjApz3kUYpD3cfbeNO1Fc1Ri9bnXs1CREgIZw79Rl40=,UjApz3kUYpD3cfbeNO1Fc1Ri9bnXs1CREgIZw79Rl40=,UjApz3kUYpD3cfbeNO1Fc1Ri9bnXs1CREgIZw79Rl40=,NaN,NaN,NaN,3691,4845,2013-07-21,tr,tr,RT @Aknc14531071: Selam olsun Peygamberin Ordu...,2020-01-17 18:28,Twitter for Android,NaN,NaN,NaN,True,NaN,1.218205e+18,absent,absent,0,0,0,0,[],[],['937037063895044097'],0.0
204120,1218900278814089227,4HST14R13t8Djig+4Kyz0VSE0KisfLFj2ONYltDBLCo=,4HST14R13t8Djig+4Kyz0VSE0KisfLFj2ONYltDBLCo=,4HST14R13t8Djig+4Kyz0VSE0KisfLFj2ONYltDBLCo=,"İstanbul, Türkiye",VUK 213/359,NaN,189,331,2019-11-25,tr,tr,"""Kopyala yapıştır"" şeklinde bir maliye raporu ...",2020-01-19 14:17,Twitter Web Client,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0,0,0,0,[],[],"['3160777371', '705374632451383296']",0.0
748193,1218790614487261184,1141259785133858816,ℂ✯MEHMET REİS ℂ✯,vali1453,NaN,R҉E҉I҉S҉ T҉I҉M҉ G҉R҉U҉P҉L҉A҉R҉I҉ G҉R҉U҉P҉ B҉A҉...,NaN,6048,5932,2019-06-19,tr,tr,RT @TheOTTOMAN_: #BülentTuran \n\n“FETÖ kasetl...,2020-01-19 07:01,Twitter for Android,NaN,NaN,NaN,True,NaN,1.218562e+18,absent,absent,0,0,0,0,['BülentTuran'],[],['335155302'],0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675279,1217691950121476096,1141259785133858816,ℂ✯MEHMET REİS ℂ✯,vali1453,NaN,R҉E҉I҉S҉ T҉I҉M҉ G҉R҉U҉P҉L҉A҉R҉I҉ G҉R҉U҉P҉ B҉A҉...,NaN,6048,5932,2019-06-19,tr,tr,RT @_mustafa_uzun_: Gülsüm ile 2 saat vakit g...,2020-01-16 06:15,Twitter for Android,NaN,NaN,NaN,True,NaN,1.217517e+18,absent,absent,0,0,0,0,[],[],['538046963'],0.0
779202,1216364871513792512,tji3Icin7zwddIfunlgqw7rgAneczjfCFN29SBMQ=,tji3Icin7zwddIfunlgqw7rgAneczjfCFN29SBMQ=,tji3Icin7zwddIfunlgqw7rgAneczjfCFN29SBMQ=,NaN,Ayaklarımı Yolunda Sabit Kıl Allah'ım...🇹🇷 ...,NaN,4914,5935,2018-09-06,tr,und,RT @MahmudEfendi_KS: https://t.co/kdni6QOgDZ,2020-01-12 14:22,Twitter for Android,NaN,NaN,NaN,True,NaN,1.216326e+18,absent,absent,0,0,0,0,[],[],['450868635'],0.0
63242,1219670444896411651,810562804700901376,Prof Dr Ketum Revnak,hocaketum,NaN,Câhilin Yanında Tevâzu Câiz Değildir,NaN,44129,4051,2016-12-18,tr,tr,@Tanolcayy @lugmus @Jakarandaaa Pek kıymetli s...,2020-01-21 17:17,Twitter for Android,2802152492,1.219644e+18,NaN,False,NaN,NaN,absent,absent,0,2,6,0,[],[],"['2802152492', '440264596', '324921240']",0.0
843454,1218299019635675136,1041001682933637120,Hicr@n,ogundogdu55,"Samsun, Türkiye",Bizimkisi bir HAK hikayesi..EYT,NaN,6494,5115,2018-09-15,tr,und,#Geceyebirşarkı \n\nhttps://t.co/ewdgpw4nJj,2020-01-17 22:28,Twitter for Android,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0,0,5,1,['Geceyebirşarkı'],['https://youtu.be/q6augjROUvY'],[],0.0


In [ ]:
train_data = merged_data[:800000]
test_data = merged_data[800000:]

In [ ]:
# Feature info
float_features_names = [
  "follower_count",
  "following_count",
  "is_retweet",
  "quote_count",
  "reply_count",
  "like_count",
  "retweet_count",
]

In [ ]:
# Data pre-processing
tokenizer = AutoTokenizer.from_pretrained(model_path)

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/245k [00:00<?, ?B/s]

In [ ]:
def process_data(batch):
  feature_dict = tokenizer(
      batch["tweet_text"].tolist(),
      padding=True,
      truncation=True,
      return_tensors="pt",
  )

  numerical_data = batch[float_features_names].astype(float)
  feature_dict["float_features"] = torch.from_numpy(numerical_data.values).float()

  labels = batch["label"].astype(float)
  feature_dict["labels"] = torch.from_numpy(labels.values).float()

  return feature_dict

In [ ]:
class BertTweetClassifier(nn.Module):
  def __init__(self, hidden_size, dense_size, numeric_feature_size, output_size, dropout=0.1):
    super().__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size

    # Use pre-trained BERT model
    self.bert = AutoModel.from_pretrained(
        model_path,
        output_hidden_states=True,
        output_attentions=True,
    )

    for param in self.bert.parameters():
        param.requires_grad = False  # No backprop here for now

    self.weights = nn.Parameter(torch.rand(13, 1))
    self.dropout = nn.Dropout(dropout)
    self.fc1 = nn.Linear(hidden_size, dense_size)
    self.fc2 = nn.Linear(dense_size + numeric_feature_size, output_size)
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()

  def forward(self, feature_dict):
    input_ids = feature_dict["input_ids"]
    float_features = feature_dict["float_features"]

    all_hidden_states, all_attentions = self.bert(input_ids)[-2:]
    batch_size = input_ids.shape[0]

    ht_cls = torch.cat(all_hidden_states)[:, :1, :].view(13, batch_size, 1, self.hidden_size)
    atten = torch.sum(ht_cls * self.weights.view(13, 1, 1, 1), dim=[1, 3])
    atten = F.softmax(atten.view(-1), dim=0)
    feature = torch.sum(ht_cls * atten.view(13, 1, 1, 1), dim=[0, 2])

    dense_out = self.fc1(self.dropout(feature))
    activ_out = self.relu(dense_out)
    concat_layer = torch.cat((activ_out, float_features), 1)
    out = self.fc2(concat_layer)
    prediction = self.sigmoid(out)

    return prediction

In [ ]:
model = BertTweetClassifier(768, 32, 7, 1).to(device)

Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model

BertTweetClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affin

In [ ]:
# for epoch_num in range(num_epochs):
#   train_data = train_data.sample(frac=1)  # Shuffle the batches each epoch
#   for batch_num in range(0, num_examples, batch_size):
#     batch_data = train_data[batch_num : min(batch_num + batch_size, num_examples)]
#     feature_dict = process_data(batch_data)
#     predictions = np.squeeze(model(feature_dict))
#     labels = feature_dict["labels"]
#     loss = criterion(predictions, labels)
#     print(loss)

In [ ]:
import gc
gc.collect()

2844

In [ ]:
batch_size = 32
learning_rate = 1e-5
num_epochs = 3
# num_examples = train_data.shape[0]
num_examples = 100000
criterion = nn.BCELoss()


# train_dataloader = DataLoader(
#   train_data,
#   sampler = RandomSampler(train_data),
#   batch_size = batch_size,
# )

# test_dataloader = DataLoader(
#   test_data,
#   sampler = SequentialSampler(test_data),
#   batch_size = batch_size,
# )

torch.cuda.empty_cache()
model.zero_grad()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch_num in range(num_epochs):
  print(f"Epoch {epoch_num} started")
  train_data = train_data.sample(frac=1)  # Shuffle the batches each epoch
  total_epoch_loss = 0

  for batch_num in range(0, num_examples, batch_size):
    model.zero_grad()

    if batch_num % (batch_size * 50) == 0:
      print(f"Batch {int(batch_num / batch_size)} started")
    batch_data = train_data[batch_num : min(batch_num + batch_size, num_examples)]

    feature_dict = process_data(batch_data).to(device)
    predictions = np.squeeze(model(feature_dict))
    labels = feature_dict["labels"]

    loss = criterion(predictions, labels)
    total_epoch_loss += loss.item()

    loss.backward()
    optimizer.step()

    del predictions
    torch.cuda.empty_cache()

  print(f"Epoch {epoch_num} finished")
  print(total_epoch_loss)

Epoch 0 started
Batch 0 started
Batch 50 started
Batch 100 started
Batch 150 started
Batch 200 started
Batch 250 started
Batch 300 started
Batch 350 started
Batch 400 started
Batch 450 started
Batch 500 started
Batch 550 started
Batch 600 started
Batch 650 started
Batch 700 started
Batch 750 started
Batch 800 started
Batch 850 started
Batch 900 started
Batch 950 started
Batch 1000 started
Batch 1050 started
Batch 1100 started
Batch 1150 started
Batch 1200 started
Batch 1250 started
Batch 1300 started
Batch 1350 started
Batch 1400 started
Batch 1450 started
Batch 1500 started
Batch 1550 started
Batch 1600 started
Batch 1650 started
Batch 1700 started
Batch 1750 started
Batch 1800 started
Batch 1850 started
Batch 1900 started
Batch 1950 started
Batch 2000 started
Batch 2050 started
Batch 2100 started
Batch 2150 started
Batch 2200 started
Batch 2250 started
Batch 2300 started
Batch 2350 started
Batch 2400 started
Batch 2450 started
Batch 2500 started
Batch 2550 started
Batch 2600 started


In [ ]:
torch.cuda.memory_summary(device="cuda", abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 8            |        cudaMalloc retries: 13        |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   10661 MB |   10674 MB |   10036 GB |   10026 GB |\n|       from large pool |   10658 MB |   10672 MB |   10032 GB |   10021 GB |\n|       from small pool |       2 MB |       3 MB |       4 GB |       4 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |   10661 MB |   10674 MB |   10036 GB |   10026 GB |\n|       from large pool |   10658 MB |   10672 MB |

In [ ]:
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size())
    except:
        pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: astor.all_symbols is deprecated.  Please use astor.symbol_data.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: astor.treewalk is deprecated.  Please use astor.tree_walk.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: astor.codegen is deprecated.  Please use astor.code_gen.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py:151: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn("torch.distributed.reduce_op is deprecated, please use "


<class 'torch.nn.parameter.Parameter'> torch.Size([32000, 768])
<class 'torch.nn.parameter.Parameter'> torch.Size([512, 768])
<class 'torch.nn.parameter.Parameter'> torch.Size([2, 768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768, 768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([3072, 768])
<class 'torch.nn.parameter.Parameter'> torch.Size([3072])
<class 'torch.nn.parameter.Parameter'> torch.Size([768, 3072])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([768])
<class 'torch.nn.parameter.Parameter'> torch.Size([3072, 768])
<class 'torch.nn.parameter.Parameter'> torch.Size([3072])
<class 'torch.nn.parameter.Parameter'> torch.Size([768, 3072])
<class 'torch.nn.parameter.Parameter'> tor